<a href="https://colab.research.google.com/github/soulsharp/Triton_kernels_ViT/blob/main/Learn_Triton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00


In [ ]:
import triton
import triton.language as tl
import torch

In [ ]:
x = torch.randn(32, 64)

In [ ]:
@triton.jit
def layer_norm_forward(
    input_ptr, # Pointer to input, 2D
    output_ptr, # Pointer to output, 2D
    weight_ptr, # Pointer to weights, shape : (num_cols, 1)
    bias_ptr, # Pointer to biases, shape : (num_cols, 1)
    mean_vector_ptr, # Row-wise mean, shape:(num_rows, 1)
    rstd_ptr, # Row-wise reciprocal of standard deviation, shape:(num_rows, 1)
    row_stride,
    num_cols,
    epsilon, # For numerical stability
    BLOCK_SIZE:tl.constexpr,
  ):

  pid = tl.program_id(0)
  input_ptr += pid * row_stride
  output_ptr = pid * row_stride

  for offset in range(0, num_cols, BLOCK_SIZE):
    offsets = offset + tl.arange(0, BLOCK_SIZE)
    mask = offsets < num_cols
    row_segment = tl.load(input_ptr + offset, mask=mask, other=0.0)
    mean_segment = tl.sum(row_segment, axis=0)

  row_mean = tl.sum(mean_segment, axis=0)

  row = tl.load(input_ptr + offsets)